# Multitask learning capability evaluation

Random forests performed well both as classifiers and as regressors, with the descripor based features (Mordred and RDKit) performing the best. Can a multitask model with imputation or a graph neural net do better?

In [1]:
import deepchem as dc
import numpy as np
import pandas as pd
import optuna
from functools import reduce

import cytoxnet.dataprep.io as io
import cytoxnet.dataprep.dataprep as dataprep
import cytoxnet.dataprep.featurize as feat
from cytoxnet.models.models import ToxModel
import cytoxnet.models.opt as opt

## Contents
This notebook contains en evaluation of the optimized models for classification tasks. The overall goal was to leverage all of our datasets (see data report section) in order to produce the best possible classifier of toxicity in a single test microbe, in this case algea. Continuous toxicity metrics are binirized by labeling the most toxic 90% of each dataset as toxic. For this reason precision score is a more valuable metric than recal score.

Models tested:
- RFC w/ RDKitDescriptors baseline: train on algea data alone
- RFC w/ RDKitDescriptor imputed with mean: train on all species (originally sparse)
- RFC w/ RDKitDescriptor imputed with iterative imputer: train on all species (originally sparse)
- GCNN w/ 0.0 wighted sparse data: train on all species (sparse)

## Create the datasets to use
Multitask learning would have the most benefit for small target datasets, so we will use the smallest in the package (Lunghini algea data) as the ultimate goal

In [2]:
## !!!!!!temporary until database query works
fish = io.load_data('../database/fish.csv', cols=['smiles', 'fish_LC50'])
daphnia = io.load_data('../database/daphnia.csv', cols=['smiles', 'daphnia_EC50'])
algea = io.load_data('../database/algea.csv', cols=['smiles', 'algea_EC50'])
rat  = io.load_data('../database/rat.csv', cols=['smiles', 'rat_LD50'])
ecoli  = io.load_data('../database/ecoli.csv', cols=['smiles', 'ecoli_MIC'])

raw = reduce(
    lambda x, y: pd.merge(x, y, how='outer', on = 'smiles'),
    [fish, daphnia, algea, rat, ecoli]
)
multitask_names = [
    'fish_LC50',
    'daphnia_EC50',
    'algea_EC50',
    'rat_LD50',
    'ecoli_MIC'
]

In [3]:
raw.describe()

,fish_LC50,daphnia_EC50,algea_EC50,rat_LD50,ecoli_MIC
count,2211.000000,2143.000000,1444.000000,7393.000000,5271.000000
mean,2.156074,1.523104,2.457666,-2.544144,2.840188
std,2.710465,2.795524,2.350359,0.958268,2.364505
min,-8.947976,-10.724468,-7.836625,-10.207000,-11.042922
25%,0.569557,0.066566,1.162368,-3.035000,1.832581
50%,2.225704,1.916923,2.708050,-2.367000,3.465736
75%,3.947383,3.502550,4.033795,-1.856000,4.158883
max,10.537415,10.126631,9.118225,0.343000,9.433484


add features - RDKit for RFCs, graphs for graph cnn

In [4]:
data_f = feat.add_features(raw, method='RDKitDescriptors', codex='../database/compounds.csv')
data_f = feat.add_features(data_f, method='ConvMolFeaturizer')

identify an independant algea test set by index

In [5]:
algea_only = data_f[~data_f.isna()['algea_EC50']]
algea_index = algea_only.index
test_index = algea_only.sample(frac=.2, random_state=0).index
baseline_index = algea_only.drop(index=test_index).index

## Baseline model
As a baseline we are using random forest classifier, which we know is capable for single tasks.

In [6]:
baseline_frame = data_f.loc[algea_index]

In [7]:
baseline_frame = dataprep.binarize_targets(baseline_frame, target_cols=['algea_EC50'], percentile=.9)

In [8]:
# create the dataset
baseline = dataprep.convert_to_dataset(
    baseline_frame,
    X_col='RDKitDescriptors',
    y_col=[
        'algea_EC50'
    ]
)

In [9]:
# split out dev and test
baseline_test = baseline.select(np.isin(baseline.ids, test_index))
baseline_dev = baseline.select(np.isin(baseline.ids, baseline_index))

retrieve hyperparmeter optimization

In [10]:
baseline_study = optuna.load_study(
    study_name='opt',
    storage="sqlite:///classification/baseline_c.db"
)

In [11]:
baseline_results = baseline_study.trials_dataframe()
baseline_results

,number,value,datetime_start,datetime_complete,duration,params_criterion,params_max_depth,params_max_features,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.902879,2021-06-01 14:36:35.577049,2021-06-01 14:36:40.581215,0 days 00:00:05.004166,gini,45.0,sqrt,8,10,225,COMPLETE
1,1,0.909396,2021-06-01 14:36:35.577139,2021-06-01 14:36:39.563972,0 days 00:00:03.986833,entropy,45.0,sqrt,1,4,195,COMPLETE
2,2,0.904460,2021-06-01 14:36:35.577589,2021-06-01 14:36:38.643183,0 days 00:00:03.065594,gini,40.0,auto,7,4,30,COMPLETE
3,3,0.902165,2021-06-01 14:36:35.577392,2021-06-01 14:36:38.604757,0 days 00:00:03.027365,entropy,5.0,auto,4,5,90,COMPLETE
4,4,0.903548,2021-06-01 14:36:35.577500,2021-06-01 14:36:38.722385,0 days 00:00:03.144885,entropy,NaN,log2,5,3,130,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
555,555,0.913974,2021-06-01 15:11:37.425130,2021-06-01 15:11:39.262270,0 days 00:00:01.837140,gini,25.0,auto,1,4,10,COMPLETE
556,556,0.913418,2021-06-01 15:11:37.994200,2021-06-01 15:11:39.656718,0 days 00:00:01.662518,gini,45.0,auto,1,4,10,COMPLETE
557,557,0.918903,2021-06-01 15:11:38.448374,2021-06-01 15:11:40.084326,0 days 00:00:01.635952,gini,25.0,auto,1,4,10,COMPLETE
558,558,0.910264,2021-06-01 15:11:38.896578,2021-06-01 15:11:40.373885,0 days 00:00:01.477307,gini,45.0,auto,1,4,10,COMPLETE


In [12]:
baseline_params = baseline_study.best_params

In [50]:
baseline_params

{'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 10}

In [51]:
baseline_study.best_value

0.9236651223187223

train the baseline model with best identified hyperparameters

In [13]:
baseline_model = ToxModel('RFC', **baseline_params)

In [14]:
baseline_model.fit(baseline_dev)

In [15]:
baseline_model.evaluate(baseline_test, ['precision_score', 'recall_score'], untransform=False, n_classes=2)

{0: 0.9064748201438849, 1: 0.972972972972973}

> The final precision score is 0.906

In [15]:
baseline_model.visualize('roc', baseline_test)

alt.LayerChart(...)

## Evaluate the multitask model
Random forests require imputation for sparse datasets, try a few methods: mean imputation, interpolation, and RFR interpolation. Additionally attempt graphs with weights.

In [16]:
import sklearn.impute

#### Impute by mean

In [17]:
mean = data_f.copy()

First do the imputation and prepare the dataset

In [18]:
mean[multitask_names] = sklearn.impute.SimpleImputer().fit_transform(
    mean[multitask_names].values
)

In [19]:
mean = dataprep.binarize_targets(mean, target_cols=multitask_names, percentile=.9)

In [20]:
mean_set = dataprep.convert_to_dataset(
    mean,
    X_col='RDKitDescriptors',
    y_col=multitask_names
)

In [21]:
mean_test = mean_set.select(np.isin(mean_set.ids, test_index))
mean_dev = mean_set.select(~np.isin(mean_set.ids, test_index))

retrieve the best paramaters for this model found via hyperparameter optimization

In [54]:
mean_study = optuna.load_study(
    study_name='opt',
    storage="sqlite:///classification/mean_c.db"
)

In [55]:
mean_results = mean_study.trials_dataframe()
mean_results

,number,value,datetime_start,datetime_complete,duration,params_criterion,params_max_depth,params_max_features,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.956450,2021-06-01 14:35:50.585763,2021-06-01 14:36:22.931778,0 days 00:00:32.346015,gini,40.0,log2,7,4,135,COMPLETE
1,1,0.956504,2021-06-01 14:35:50.586888,2021-06-01 14:36:39.383606,0 days 00:00:48.796718,entropy,25.0,log2,3,7,105,COMPLETE
2,2,0.956450,2021-06-01 14:35:50.643721,2021-06-01 14:38:15.368856,0 days 00:02:24.725135,entropy,10.0,sqrt,6,6,220,COMPLETE
3,3,0.956450,2021-06-01 14:35:50.719532,2021-06-01 14:36:04.471369,0 days 00:00:13.751837,entropy,NaN,sqrt,8,8,15,COMPLETE
4,4,0.956513,2021-06-01 14:35:50.727065,2021-06-01 14:36:10.244654,0 days 00:00:19.517589,gini,40.0,log2,4,10,70,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
275,275,0.957969,2021-06-01 14:42:09.000975,2021-06-01 14:42:27.873074,0 days 00:00:18.872099,entropy,25.0,sqrt,1,3,25,COMPLETE
276,276,0.958027,2021-06-01 14:42:09.589562,2021-06-01 14:42:28.584048,0 days 00:00:18.994486,entropy,40.0,sqrt,1,3,25,COMPLETE
277,277,0.958191,2021-06-01 14:42:11.716209,2021-06-01 14:42:30.697741,0 days 00:00:18.981532,entropy,40.0,auto,1,3,25,COMPLETE
278,278,0.957891,2021-06-01 14:43:23.897917,2021-06-01 14:43:34.736957,0 days 00:00:10.839040,entropy,40.0,auto,1,3,15,COMPLETE


In [56]:
mean_params = mean_study.best_params

In [57]:
mean_params

{'criterion': 'entropy',
 'max_depth': 25,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 10}

In [58]:
mean_study.best_value

0.9598840528678435

train the multitask model using the best found parameters

In [59]:
mean_model = ToxModel('RFC', transformers=[], tasks=multitask_names, **mean_params)

In [60]:
mean_model.fit(mean_dev)

In [61]:
mean_model.evaluate(
    mean_test, ['precision_score', 'recall_score'],
    untransform=False,
    per_task_metrics=True,
    n_classes=2)

({0: 0.8133695923078992, 1: 0.8001910376515406},
 {0: [0.7615384615384615,
   0.9444444444444444,
   0.45454545454545453,
   0.9098039215686274,
   0.9965156794425087],
  1: [0.9166666666666666,
   0.19767441860465115,
   0.9765625,
   0.9169960474308301,
   0.9930555555555556]})

> The precision score for the independant algea test set is. 0.4545

In [62]:
mean_model.visualize('roc', mean_test, task='algea_EC50')

alt.LayerChart(...)

### Impute by interpolation

In [28]:
from sklearn.experimental import enable_iterative_imputer

First impute and prepare the data. This requires sklearns  experimental imputer.

In [29]:
iterpute = data_f.copy()

In [30]:
iterpute[multitask_names] = sklearn.impute.IterativeImputer(random_state=0).fit_transform(
    iterpute[multitask_names].values
)

In [31]:
iterpute = dataprep.binarize_targets(iterpute, target_cols=multitask_names, percentile=.9)

In [32]:
iterpute_set = dataprep.convert_to_dataset(
    iterpute,
    X_col='RDKitDescriptors',
    y_col=multitask_names
)

In [33]:
iterpute_test = iterpute_set.select(np.isin(iterpute_set.ids, test_index))
iterpute_dev = iterpute_set.select(~np.isin(iterpute_set.ids, test_index))

Retrieve hyperparameter optimization for this model

In [63]:
iterpute_study = optuna.load_study(
    study_name='opt',
    storage="sqlite:///classification/inter_c.db"
)

In [64]:
iterpute_results = iterpute_study.trials_dataframe()
iterpute_results

,number,value,datetime_start,datetime_complete,duration,params_criterion,params_max_depth,params_max_features,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.908367,2021-06-01 14:35:50.106947,2021-06-01 14:36:58.413702,0 days 00:01:08.306755,gini,20.0,auto,8,2,170,COMPLETE
1,1,0.909630,2021-06-01 14:35:50.192340,2021-06-01 14:37:55.282279,0 days 00:02:05.089939,gini,20.0,sqrt,2,4,265,COMPLETE
2,2,0.909222,2021-06-01 14:35:50.124678,2021-06-01 14:37:58.144892,0 days 00:02:08.020214,entropy,45.0,auto,4,5,140,COMPLETE
3,3,0.908374,2021-06-01 14:35:50.165795,2021-06-01 14:36:24.418858,0 days 00:00:34.253063,gini,5.0,sqrt,5,5,170,COMPLETE
4,4,0.909493,2021-06-01 14:35:51.078229,2021-06-01 14:37:37.457712,0 days 00:01:46.379483,gini,15.0,auto,2,5,260,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
275,275,0.914962,2021-06-01 14:45:12.745411,2021-06-01 14:45:17.802814,0 days 00:00:05.057403,gini,40.0,sqrt,1,3,10,COMPLETE
276,276,0.912154,2021-06-01 14:45:17.996689,2021-06-01 14:45:27.427344,0 days 00:00:09.430655,gini,40.0,sqrt,1,3,20,COMPLETE
277,277,0.914276,2021-06-01 14:45:27.624391,2021-06-01 14:45:32.761593,0 days 00:00:05.137202,gini,40.0,sqrt,1,3,10,COMPLETE
278,278,0.914784,2021-06-01 14:45:29.413000,2021-06-01 14:45:34.651221,0 days 00:00:05.238221,gini,40.0,sqrt,1,3,10,COMPLETE


In [36]:
iterpute_params = iterpute_study.best_params

In [65]:
iterpute_params

{'criterion': 'gini',
 'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 10}

In [66]:
iterpute_study.best_value

0.9157152387746089

train the model using the hyperparameters found best for this model on the development set.

In [37]:
iterpute_model = ToxModel('RFC', transformers=[], tasks=multitask_names, **iterpute_params)

In [38]:
iterpute_model.fit(iterpute_dev)

In [39]:
iterpute_model.evaluate(iterpute_test, ['precision_score', 'recall_score'], untransform=False, per_task_metrics=True, n_classes=2)

({0: 0.7739864178508946, 1: 0.9355953128139383},
 {0: [0.797752808988764,
   0.7165354330708661,
   0.4770992366412214,
   0.9066147859922179,
   0.9719298245614035],
  1: [0.9508928571428571,
   0.914572864321608,
   0.9057971014492754,
   0.9209486166007905,
   0.9857651245551602]})

> The final precision score for multitask RFC with interpolated imputation is 0.4771

In [39]:
iterpute_model.visualize('roc', iterpute_test, task='algea_EC50')

alt.LayerChart(...)

## Try a Graph multitask

In this case, instead of imputation, the neural network architectures can accept a weight matrix in the same shape as the targets, thus sparse data can be masked out. 

In [40]:
graph = data_f.copy()

Prepare the data. During binarizing, we weight all targets originally without data as 0.0.

In [41]:
graph = dataprep.binarize_targets(graph, target_cols=multitask_names, percentile = .9)

In [42]:
graph_set = dataprep.convert_to_dataset(
    graph,
    X_col='ConvMolFeaturizer',
    y_col=multitask_names,
    w_label='w'
)

In [43]:
graph_test = graph_set.select(np.isin(graph_set.ids, test_index))
graph_dev = graph_set.select(~np.isin(graph_set.ids, test_index))

Retrieve best hyperparameters for the graph multitask model.

In [44]:
graph_study = optuna.load_study(
    study_name='opt',
    storage="sqlite:///classification/graph_c.db"
)

In [45]:
graph_results = graph_study.trials_dataframe()
graph_results

/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [32] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution 

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_dense_layer_size,params_dropout,params_graph_conv_layers,params_number_atom_features,system_attrs_fail_reason,state
0,0,NaN,2021-06-01 16:01:07.242437,NaT,NaT,100,208,0.360900,"[128, 128, 128]",25,NaN,RUNNING
1,1,NaN,2021-06-01 16:07:16.202917,NaT,NaT,150,232,0.358572,[64],100,NaN,RUNNING
2,2,NaN,2021-06-01 16:07:30.164426,NaT,NaT,300,316,0.397522,"[32, 32]",125,NaN,RUNNING
3,3,NaN,2021-06-01 16:10:37.815782,NaT,NaT,250,208,0.340126,"[32, 32]",75,NaN,RUNNING
4,4,NaN,2021-06-01 16:22:06.417704,NaT,NaT,275,196,0.206110,"[128, 128]",25,NaN,RUNNING
...,...,...,...,...,...,...,...,...,...,...,...,...
167,167,-0.206107,2021-06-01 20:42:42.867552,2021-06-01 20:47:38.290639,0 days 00:04:55.423087,100,88,0.000975,[32],25,NaN,COMPLETE
168,168,-0.310806,2021-06-01 20:43:41.501866,2021-06-01 20:49:19.633610,0 days 00:05:38.131744,50,76,0.001134,[32],25,NaN,COMPLETE
169,169,-0.606919,2021-06-01 20:47:38.368857,2021-06-01 20:54:29.265034,0 days 00:06:50.896177,100,208,0.368068,[32],25,NaN,COMPLETE
170,170,-0.455734,2021-06-01 20:49:19.714814,2021-06-01 20:57:58.620988,0 days 00:08:38.906174,100,304,0.056853,[32],25,NaN,COMPLETE


In [46]:
graph_params = graph_study.best_params

/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [32] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution 

In [67]:
graph_params

{'batch_size': 275,
 'dense_layer_size': 112,
 'dropout': 0.013039897074701816,
 'graph_conv_layers': [64],
 'number_atom_features': 25}

In [68]:
graph_study.best_value

/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [32] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128] which is of type list.
  warnings.warn(message)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/optuna/distributions.py:430: UserWarning: Choices for a categorical distribution 

-0.10823066196246194

Train the graph model using the best parameters on the entire dev set.

In [47]:
graph_model = ToxModel(
    'GraphCNN',
    tasks=multitask_names,
    transformers=[],
    mode='classification',
    **graph_params
)

In [48]:
graph_model.fit(graph_dev, nb_epoch=50)

/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool/Reshape_14:0", shape=(1424,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool/Reshape_13:0", shape=(1424, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool/Reshape_17:0", shape=(6010,), dtype=int32), values=Tensor("gradient_tape/private__grap

0.020670228004455567

In [66]:
graph_model.evaluate(
    graph_test,
    ['precision_score', 'recall_score'],
    untransform=False,
    use_sample_weights=True,
    per_task_metrics=True,
    n_classes=2
)

/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


({0: 0.75514324272219, 1: 0.7678831314208416},
 {0: [0.9538461538461539, 0.9824561403508771, 0.9194139194139194, 0.92, 0.0],
  1: [0.9393939393939394,
   0.9824561403508771,
   0.9691119691119691,
   0.9484536082474226,
   0.0]})

> The precision score for the test set is 0.919. This is approximately 1% better than the baseline single task model.

In [49]:
graph_model.visualize('roc', graph_test, task='algea_EC50')

alt.LayerChart(...)